# Introduction to the Project

In my chosen project, I went into the world of machine learning by concentrating on Neural Networks in particular. The MNIST dataset, obtained from its official source, served as the foundation for my experiments and model building. This dataset, which included both training and assessment sets, served as a solid foundation for training and refining my machine learning model.

TensorFlow, a strong Python framework designed for machine learning applications, became the foundation of my project. Its comprehensive set of tools and features provided invaluable assistance in establishing, fine-tuning, and optimising the neural network model. TensorFlow's user-friendly interface allowed me to navigate through sophisticated neural network designs with relative ease, allowing me to focus on the model's complexities rather than implementation challenges.

Throughout this project, I became engaged in the subtleties of neural networks, learning about their layers, activations, and optimisation strategies. The ongoing process of training and refining the model with the MNIST dataset provides significant insights into machine learning's inner workings.

Overall, this project provided as a thorough examination of neural networks, aided by TensorFlow's essential assistance, allowing me to navigate the realm of machine learning and get practical expertise in creating and refining models for image recognition tasks.

# Imports

In [1]:
from numpy.random import seed
seed(888)
from tensorflow import random
random.set_seed(404)

In [2]:
import os
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from time import  strftime
from PIL import Image

Instructions for updating:
non-resource variables are not supported in the long term


# Constants

In [3]:
X_TRAIN_PATH = 'MINIST/digit_xtrain.csv'
X_TEST_PATH = 'MINIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MINIST/digit_ytrain.csv'
Y_TEST_PATH = 'MINIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

# Get the Data

In [4]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 1.41 ms, sys: 1.05 ms, total: 2.46 ms
Wall time: 2.01 ms


In [5]:
y_train_all.shape

(60000,)

In [6]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [7]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 547 ms, sys: 61.3 ms, total: 608 ms
Wall time: 639 ms


In [8]:
%%time

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

CPU times: user 92.1 ms, sys: 11.9 ms, total: 104 ms
Wall time: 108 ms


# Explore

In [9]:
x_train_all.shape

(60000, 784)

In [10]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [11]:
y_train_all.shape

(60000,)

In [12]:
y_test.shape

(10000,)

In [13]:
x_test.shape

(10000, 784)

In [14]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Data Preprocessing

In [15]:
# Re-scale
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0

### One hot encoding

In [16]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [17]:
y_train_all.shape

(60000, 10)

In [18]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

## Create Validation from Training Data

In [19]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [20]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [21]:
x_val.shape

(10000, 784)

In [22]:
x_train.shape

(50000, 784)

# Setup Tensorflow Graph

In [23]:
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

### Neural Network Acrhitecture

### Hyperparameters

In [24]:
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [25]:
def setup_layer(input, weight_dim, bias_dim, name):

    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b

        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)

        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)

        return layer_out

In [26]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1],
                      bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2],
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES],
                      bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Tensorboard Setup


In [27]:
# Folder for Tensorboard

folder_name = f'{model_name} at {strftime("%H.%M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Successfully created directories!')

Successfully created directories!


# Loss optimization amd Metrics

### Defining Loss Function

In [28]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

### Defining Optimizer

In [29]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

### Accuracy Metric

In [30]:
with tf.name_scope('accuracy_calc'):
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [31]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

#### Check Input Images in Tensorboard

In [32]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session


In [33]:
sess = tf.Session()

### Setup Filewriter and Merge Summaries

In [34]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation')

### Intialize all the varaibles

In [35]:
init = tf.global_variables_initializer()
sess.run(init)

2023-12-01 06:07:51.787160: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled


### Batching

In [36]:
size_of_batch = 1000

In [37]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [38]:
def next_batch(batch_size, data, labels):

    global num_examples
    global index_in_epoch

    start = index_in_epoch
    index_in_epoch += batch_size

    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size

    end = index_in_epoch

    return data[start:end], labels[start:end]

### Training Loop

In [39]:
for epoch in range(nr_epochs):

    for i in range(nr_iterations):

        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)

        feed_dictionary = {X:batch_x, Y:batch_y}

        sess.run(train_step, feed_dict=feed_dictionary)


    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)

    train_writer.add_summary(s, epoch)

    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')

    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)

print('Done training!')

Epoch 0 	| Training Accuracy = 0.8500000238418579
Epoch 1 	| Training Accuracy = 0.8640000224113464
Epoch 2 	| Training Accuracy = 0.8629999756813049
Epoch 3 	| Training Accuracy = 0.8659999966621399
Epoch 4 	| Training Accuracy = 0.8730000257492065
Epoch 5 	| Training Accuracy = 0.8769999742507935
Epoch 6 	| Training Accuracy = 0.9660000205039978
Epoch 7 	| Training Accuracy = 0.9779999852180481
Epoch 8 	| Training Accuracy = 0.9760000109672546
Epoch 9 	| Training Accuracy = 0.9800000190734863
Epoch 10 	| Training Accuracy = 0.9869999885559082
Epoch 11 	| Training Accuracy = 0.9860000014305115
Epoch 12 	| Training Accuracy = 0.9869999885559082
Epoch 13 	| Training Accuracy = 0.9869999885559082
Epoch 14 	| Training Accuracy = 0.9869999885559082
Epoch 15 	| Training Accuracy = 0.9879999756813049
Epoch 16 	| Training Accuracy = 0.9890000224113464
Epoch 17 	| Training Accuracy = 0.9879999756813049
Epoch 18 	| Training Accuracy = 0.9909999966621399
Epoch 19 	| Training Accuracy = 0.9890000

# Make Prediction

In [40]:
img = Image.open('MINIST/test_img.png')
img

In [41]:
bw = img.convert('L')

In [42]:
img_array = np.invert(bw)
img_array.shape

(28, 28)

In [43]:
test_img = img_array.ravel()

In [44]:
test_img.shape

(784,)

In [45]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})

In [46]:
print(f'Prediction for test image is {prediction}')

Prediction for test image is [2]


# Testing and Evalulation

In [47]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy on test set is 97.73%


# Reset for the Next run

In [48]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()

# Reductant Code

In [49]:
# with tf.name_scope('first_hidden_layer'):

#     initial_w1 = tf.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.1, seed=42)
#     w1 = tf.Variable(initial_value=initial_w1, name='W1')

#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value=initial_b1, name='B1')

#     layer1_in = tf.matmul(X, w1) + b1
#     layer1_out = tf.nn.relu(layer1_in)

In [50]:
# with tf.name_scope('second_hidden_layer'):

#     initial_w2 = tf.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
#     w2 = tf.Variable(initial_value=initial_w2, name='W2')

#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value=initial_b2, name='B2')

#     layer2_in = tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

In [51]:
# with tf.name_scope('output_layer'):

#     initial_w3 = tf.truncated_normal(shape=[n_hidden2, NR_CLASSES], stddev=0.1, seed=42)
#     w3 = tf.Variable(initial_value=initial_w3, name='W3')

#     initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES])
#     b3 = tf.Variable(initial_value=initial_b3, name='B3')

#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     output = tf.nn.softmax(layer3_in)

In [52]:
# b3.eval(sess)

In [53]:
# Model without drop out

# layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1],
#                      bias_dim=[n_hidden1], name='layer_1')

# layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2],
#                      bias_dim=[n_hidden2], name='layer_2')

# output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES],
#                      bias_dim=[NR_CLASSES], name='out')

# model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{nr_epochs}'